# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - SQLite

**Subtopic** - Python SQLite CRUD Operations

**Use Case** - Implement a Python module that manages SQLite schema evolution by automatically applying migrations based on a versioning system stored in a dedicated table.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

## Problem Overview

You are tasked with implementing a Python module that **automatically manages bidirectional schema evolution** of an SQLite database using a versioning system. Each migration is tied to a **specific version number** and must support both **upgrade** and **downgrade** logic.

To track the schema version, the system uses a special table called `schema_version`, which contains a single row with the current version number. If the table does not exist, the schema is assumed to be at **version 0**.

Your system must:

* Automatically determine the current version of the schema.
* **Upgrade** or **downgrade** the schema to match a user-specified **target version**.
* Execute migrations **in correct order and direction** (ascending for upgrade, descending for downgrade).
* Roll back any partial changes if a migration fails.
* Guarantee idempotency, atomicity, and version integrity.


## Input Format

* A path to the SQLite database file as a string.
* A dictionary mapping integers (version numbers) to migration steps.
* A target version (integer) to migrate to.

Each migration step is a dictionary with two required keys:

* `"upgrade"`: A function to migrate *to* this version.
* `"downgrade"`: A function to revert *from* this version.

**Input Parameters:**

```python
database_path: str
migrations: dict[int, dict[str, Callable[[sqlite3.Connection], None]]]
target_version: int
```


## Output Format

* After applying the necessary migrations, the function should return the **new schema version**, which must be equal to `target_version`.

**Output:**

```python
new_schema_version: int
```


## Examples

### Example 1 - Upgrade from 0 -> 2

```python
database_path = "my_database.db"
migrations = {
    1: {
        "upgrade": lambda conn: conn.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT)"),
        "downgrade": lambda conn: conn.execute("DROP TABLE users")
    },
    2: {
        "upgrade": lambda conn: conn.execute("ALTER TABLE users ADD COLUMN email TEXT"),
        "downgrade": lambda conn: conn.execute("CREATE TABLE tmp AS SELECT id, name FROM users; DROP TABLE users; CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT); INSERT INTO users SELECT id, name FROM tmp; DROP TABLE tmp")
    }
}

target_version = 2
# Output: 2
```

### Example 2 - Downgrade from 2 -> 1

```python
# Assume database is currently at version 2
target_version = 1

# Output: 1

# Explanation:
# Only the downgrade for version 2 is applied in this case.
```

# Requirements



## Explicit Requirements

* Create the `schema_version` table if it doesn't exist.
* Enforce a single-row invariant in `schema_version`.
* Migrations must be applied **in order**:

  * Ascending (for upgrade).
  * Descending (for downgrade).
* A migration's version is considered **applied after the upgrade function finishes successfully**.
* A migration's version is considered **rolled back after the downgrade function finishes successfully**.
* Use transactions to **ensure atomicity**—a failure in any migration must roll back that entire step.
* Update the `schema_version` row **only after** the migration step succeeds.


## Implicit Requirements

* Migration functions must be idempotent when applied exactly once.
* Do not skip or re-apply migrations that are already at the correct version.
* Do not downgrade below version 0.
* Downgrades must not silently truncate data (unless explicitly written into the migration).
* Must be safe to run multiple times against an already-upgraded or downgraded schema.
* Must not leave the database in an inconsistent state after failure.


## Edge Cases

* Empty `migrations` dict: return current version unchanged.
* `target_version == current_version`: no-op.
* Migration version missing either `upgrade` or `downgrade`: raise error.
* Migration fails: transaction must rollback, version must remain unchanged.
* Attempt to downgrade below version 0: raise error.
* Migrations are non-consecutive: apply available ones correctly.


## Function Signature

```python
def apply_schema_migrations(
    database_path: str,
    migrations: dict[int, dict[str, Callable[[sqlite3.Connection], None]]],
    target_version: int
) -> int:
    pass
```


## Constraints

* 1 <= number of migration entries <= 100
* 0 <= version number <= 1000
* Migration versions must be unique integers
* Each migration must include both `upgrade` and `downgrade` keys
* The database file may or may not exist
* Python version >= 3.7

In [ ]:
# code

import sqlite3
from typing import Callable


def apply_schema_migrations(
    database_path: str,
    migrations: dict[int, dict[str, Callable[[sqlite3.Connection], None]]],
    target_version: int
) -> int:
    """
    Applies schema migrations to reach the target version.

    Parameters:
        database_path (str): Path to the SQLite database file.
        migrations (dict): Dictionary mapping version numbers to a dictionary
            containing "upgrade" and "downgrade" functions.
        target_version (int): The desired schema version.

    Returns:
        int: The new schema version after successful migrations.

    Raises:
        RuntimeError: If a migration fails or the schema_version table is invalid.
        ValueError: If a required migration direction is missing.
    """

    conn = sqlite3.connect(database_path)
    try:
        # Ensure schema_version table exists
        conn.execute("""
            CREATE TABLE IF NOT EXISTS schema_version (
                version INTEGER PRIMARY KEY CHECK(version >= 0)
            )
        """)
        conn.commit()

        # Enforce single-row invariant
        cursor = conn.execute("SELECT version FROM schema_version")
        rows = cursor.fetchall()
        if len(rows) > 1:
            raise RuntimeError("schema_version table contains multiple rows.")

        if not rows:
            current_version = 0
            with conn:
                conn.execute("INSERT INTO schema_version (version) VALUES (0)")
        else:
            current_version = rows[0][0]

        if current_version == target_version:
            return current_version

        def update_version(new_version: int) -> None:
            conn.execute("DELETE FROM schema_version")
            conn.execute(
                "INSERT INTO schema_version (version) VALUES (?)",
                (new_version,)
            )

        if target_version > current_version:
            # Upgrade
            for version in range(current_version + 1, target_version + 1):
                step = migrations.get(version)
                if not step or "upgrade" not in step:
                    raise ValueError(
                        f"No upgrade function provided for version {version}"
                    )

                try:
                    with conn:
                        step["upgrade"](conn)
                        update_version(version)
                except Exception as e:
                    raise RuntimeError(
                        f"Failed to upgrade to version {version}: {e}"
                    ) from e

        else:
            # Downgrade
            for version in range(current_version, target_version, -1):
                step = migrations.get(version)
                if not step or "downgrade" not in step:
                    raise ValueError(
                        f"No downgrade function provided for version {version}"
                    )

                try:
                    with conn:
                        step["downgrade"](conn)
                        update_version(version - 1)
                except Exception as e:
                    raise RuntimeError(
                        f"Failed to downgrade from version {version}: {e}"
                    ) from e

        return target_version

    finally:
        conn.close()

In [ ]:
# tests

import unittest
import sqlite3
import os
import tempfile
from main import apply_schema_migrations


class TestSchemaMigrations(unittest.TestCase):
    """Unit test suite for schema migration system using
      apply_schema_migrations."""

    def setUp(self):
        """Create a temporary SQLite file before each test."""
        self.temp_file = tempfile.NamedTemporaryFile(
            delete=False, suffix=".sqlite")
        self.db_path = self.temp_file.name
        self.temp_file.close()

    def tearDown(self):
        """Clean up the temporary database file."""
        try:
            os.remove(self.db_path)
        except PermissionError:
            import gc
            gc.collect()
            try:
                os.remove(self.db_path)
            except Exception:
                pass

    def test_create_schema_version_on_fresh_db(self):
        """Should create schema_version on empty DB and upgrade to
        version 1."""
        migrations = {
            1: {
                "upgrade": lambda conn: conn.execute(
                    "CREATE TABLE test (id INTEGER)"),
                "downgrade": lambda conn: conn.execute("DROP TABLE test")
            }
        }
        new_version = apply_schema_migrations(self.db_path, migrations, 1)
        self.assertEqual(new_version, 1)

    def test_upgrade_multiple_versions(self):
        """Should apply all upgrades from version 0 to 3 in order."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("CREATE TABLE t1 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t1")},
            2: {"upgrade": lambda c: c.execute("CREATE TABLE t2 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t2")},
            3: {"upgrade": lambda c: c.execute("CREATE TABLE t3 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t3")},
        }
        new_version = apply_schema_migrations(self.db_path, migrations, 3)
        self.assertEqual(new_version, 3)

    def test_downgrade_single_step(self):
        """Should downgrade from version 3 to version 1."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("CREATE TABLE t1 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t1")},
            2: {"upgrade": lambda c: c.execute("CREATE TABLE t2 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t2")},
            3: {"upgrade": lambda c: c.execute("CREATE TABLE t3 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t3")},
        }
        apply_schema_migrations(self.db_path, migrations, 3)
        new_version = apply_schema_migrations(self.db_path, migrations, 1)
        self.assertEqual(new_version, 1)

    def test_noop_if_target_equals_current(self):
        """Should do nothing if target version equals current version."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("CREATE TABLE t (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t")},
        }
        apply_schema_migrations(self.db_path, migrations, 1)
        new_version = apply_schema_migrations(self.db_path, migrations, 1)
        self.assertEqual(new_version, 1)

    def test_empty_migrations_returns_current(self):
        """Should return current version if migrations dict is empty."""
        new_version = apply_schema_migrations(self.db_path, {}, 0)
        self.assertEqual(new_version, 0)

    def test_missing_upgrade_raises_error(self):
        """Should raise ValueError if upgrade function is missing."""
        migrations = {
            1: {"downgrade": lambda c: c.execute("SELECT 1")}
        }
        with self.assertRaises(ValueError):
            apply_schema_migrations(self.db_path, migrations, 1)

    def test_missing_downgrade_raises_error(self):
        """Should raise ValueError if downgrade function is missing."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("SELECT 1")}
        }
        apply_schema_migrations(self.db_path, migrations, 1)
        with self.assertRaises(ValueError):
            apply_schema_migrations(self.db_path, migrations, 0)

    def test_downgrade_below_zero_raises_error(self):
        """Should raise ValueError when trying to downgrade below version 0."""
        with self.assertRaises(ValueError):
            apply_schema_migrations(self.db_path, {}, -1)

    def test_failed_upgrade_rolls_back(self):
        """Should not apply partial upgrade on failure."""
        def broken_migration(conn):
            raise RuntimeError("Upgrade failed before any change")

        migrations = {
            1: {
                "upgrade": broken_migration,
                "downgrade": lambda c: c.execute("DROP TABLE IF EXISTS safe")
            }
        }

        with self.assertRaises(RuntimeError):
            apply_schema_migrations(self.db_path, migrations, 1)

        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute(
                "SELECT name FROM sqlite_master WHERE type='table'"
            )
            tables = [row[0] for row in cursor.fetchall()]
            self.assertNotIn("safe", tables)

    def test_idempotent_upgrade_execution(self):
        """Should allow safe re-run of same upgrade without duplication."""
        migrations = {
            1: {"upgrade": lambda c: c.execute(
                "CREATE TABLE IF NOT EXISTS t (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t")}
        }
        apply_schema_migrations(self.db_path, migrations, 1)
        new_version = apply_schema_migrations(self.db_path, migrations, 1)
        self.assertEqual(new_version, 1)

    def test_non_consecutive_versions(self):
        """Should skip missing versions if not present in migrations."""
        migrations = {
            1: {"upgrade": lambda c: None, "downgrade": lambda c: None},
            2: {"upgrade": lambda c: None, "downgrade": lambda c: None},
            3: {"upgrade": lambda c: c.execute("CREATE TABLE x (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE x")},
        }

        version = apply_schema_migrations(self.db_path, migrations, 3)
        self.assertEqual(version, 3)

    def test_multiple_downgrades(self):
        """Should downgrade multiple steps from version 3 to 0."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("CREATE TABLE t1 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t1")},
            2: {"upgrade": lambda c: c.execute("CREATE TABLE t2 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t2")},
            3: {"upgrade": lambda c: c.execute("CREATE TABLE t3 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t3")},
        }
        apply_schema_migrations(self.db_path, migrations, 3)
        new_version = apply_schema_migrations(self.db_path, migrations, 0)
        self.assertEqual(new_version, 0)

    def test_partial_migration_is_atomic(self):
        """Should rollback version if error occurs during downgrade."""
        def downgrade_with_error(conn):
            conn.execute("DROP TABLE t1")
            raise Exception("Oops")

        migrations = {
            1: {
                "upgrade": lambda c: c.execute("CREATE TABLE t1 (id INTEGER)"),
                "downgrade": downgrade_with_error
            }
        }

        apply_schema_migrations(self.db_path, migrations, 1)

        with self.assertRaises(Exception):
            apply_schema_migrations(self.db_path, migrations, 0)

        version = apply_schema_migrations(self.db_path, migrations, 1)
        self.assertEqual(version, 1)

    def test_target_version_equal_zero(self):
        """Should downgrade to version 0 from any higher version."""
        migrations = {
            1: {"upgrade": lambda c: c.execute("CREATE TABLE t1 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t1")},
            2: {"upgrade": lambda c: c.execute("CREATE TABLE t2 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t2")},
            3: {"upgrade": lambda c: c.execute("CREATE TABLE t3 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t3")},
        }
        apply_schema_migrations(self.db_path, migrations, 3)
        version = apply_schema_migrations(self.db_path, migrations, 0)
        self.assertEqual(version, 0)

    def test_max_version_boundary(self):
        """Ensure migrations are applied in correct ascending order to
        reach version 1000."""
        order = []

        migrations = {
            v: {
                "upgrade": (lambda v=v: lambda c: order.append(v))(),
                "downgrade": (lambda v=v: lambda c: order.append(-v))()
            }
            for v in range(1, 1001)
        }

        version = apply_schema_migrations(self.db_path, migrations, 1000)

        self.assertEqual(version, 1000)
        self.assertEqual(order, list(range(1, 1001)))

    def test_existing_schema_version_table(self):
        """Should respect existing schema_version table and continue
          migration."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("CREATE TABLE schema_version (version INTEGER)")
            conn.execute("INSERT INTO schema_version (version) VALUES (2)")
            conn.commit()

        migrations = {
            3: {
                "upgrade": lambda c: c.execute("CREATE TABLE t3 (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE t3")
            }
        }

        version = apply_schema_migrations(self.db_path, migrations, 3)
        self.assertEqual(version, 3)

    def test_schema_version_table_single_row(self):
        """Should raise error if schema_version table contains more than\
              one row."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("CREATE TABLE schema_version (version INTEGER)")
            conn.execute("INSERT INTO schema_version (version) VALUES (1)")
            conn.execute("INSERT INTO schema_version (version) VALUES (2)")
            conn.commit()

        migrations = {
            3: {
                "upgrade": lambda c: c.execute("CREATE TABLE x (id INTEGER)"),
                "downgrade": lambda c: c.execute("DROP TABLE x")
            }
        }

        with self.assertRaises(RuntimeError):
            apply_schema_migrations(self.db_path, migrations, 3)


if __name__ == '__main__':
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/403c2141-1cf1-4bb0-94fa-3a5b604a22f8?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67749%2Fview

#### Model code:

```python
import sqlite3
from typing import Callable, Dict

def apply_schema_migrations(
    database_path: str,
    migrations: Dict[int, Dict[str, Callable[[sqlite3.Connection], None]]],
    target_version: int
) -> int:
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    try:
        # Ensure schema_version table exists
        conn.execute('''
            CREATE TABLE IF NOT EXISTS schema_version (
                version INTEGER PRIMARY KEY CHECK(version >= 0)
            )
        ''')
        conn.commit()

        # Get the current version
        cursor = conn.cursor()
        cursor.execute('SELECT version FROM schema_version LIMIT 1') # <Issue>: The code assumes that the table has only zero or one row, but does not verify or enforce that. If there are multiple rows, this silently uses the first one.
        row = cursor.fetchone()
        current_version = row[0] if row else 0

        if current_version == target_version:
            return current_version

        # Helper function to update the schema version
        def set_schema_version(version: int):
            conn.execute('DELETE FROM schema_version')
            conn.execute('INSERT INTO schema_version (version) VALUES (?)', (version,))
            conn.commit()

        # Apply migrations in ascending order (for upgrades)
        if target_version > current_version:
            for version in range(current_version + 1, target_version + 1):
                if version in migrations and "upgrade" in migrations[version]:
                    try:
                        # Use transaction for the upgrade step
                        with conn:
                            migrations[version]["upgrade"](conn)
                            set_schema_version(version) # <Issue>: This function uses conn.commit() inside a with conn: block. This is unnecessary and may cause inconsistent transaction handling.
                    except Exception as e:
                        # Roll back on error
                        conn.rollback()
                        raise RuntimeError(f"Failed to upgrade to version {version}: {e}")
                else:
                    raise ValueError(f"No upgrade function for version {version}")

        # Apply migrations in descending order (for downgrades)
        elif target_version < current_version:
            for version in range(current_version, target_version, -1):
                if version in migrations and "downgrade" in migrations[version]: # <Issue>: This will raise KeyError if version is in migrations, but its value is None or malformed.
                    try:
                        # Use transaction for the downgrade step
                        with conn:
                            migrations[version]["downgrade"](conn)
                            set_schema_version(version - 1)
                    except Exception as e:
                        # Roll back on error
                        conn.rollback()
                        raise RuntimeError(f"Failed to downgrade from version {version}: {e}")
                else:
                    raise ValueError(f"No downgrade function for version {version}")

        return target_version

    finally:
        conn.close()
```